__Handling Missing data__

*Missing data* is when there is no feature a particular which might be the observation was not recorded or due to data corruption.

In [54]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [55]:
df = pd.read_csv("employee_data.csv")
df.head()

,employee_id,age,gender,department,years_of_experience,salary,performance_score,training_hours,job_satisfaction,last_promotion
0,1,56.0,NaN,Marketing,NaN,58195.0,4.0,14.0,NaN,10.0
1,2,46.0,Male,Sales,NaN,96192.0,7.0,30.0,3.0,0.0
2,3,32.0,Female,HR,12.0,45609.0,1.0,NaN,5.0,3.0
3,4,60.0,Female,Sales,12.0,49535.0,1.0,12.0,2.0,6.0
4,5,25.0,Female,NaN,4.0,34110.0,9.0,NaN,1.0,3.0


In [56]:
# Summary statistics
df.describe()

,employee_id,age,years_of_experience,salary,performance_score,training_hours,job_satisfaction,last_promotion
count,10000.00000,9500.000000,9000.000000,9300.000000,9500.000000,8500.000000,9000.000000,8000.000000
mean,5000.50000,41.602632,10.068000,89700.182473,5.530421,20.159882,2.993222,4.983250
std,2886.89568,13.773172,6.006926,34628.706457,2.855251,11.728353,1.412428,3.179423
min,1.00000,18.000000,0.000000,30005.000000,1.000000,0.000000,1.000000,0.000000
25%,2500.75000,30.000000,5.000000,59310.250000,3.000000,10.000000,2.000000,2.000000
50%,5000.50000,42.000000,10.000000,89744.000000,6.000000,20.000000,3.000000,5.000000
75%,7500.25000,53.000000,15.000000,119943.250000,8.000000,30.000000,4.000000,8.000000
max,10000.00000,65.000000,20.000000,149998.000000,10.000000,40.000000,5.000000,10.000000


In [57]:
# Dataset summary
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   employee_id          10000 non-null  int64  
 1   age                  9500 non-null   float64
 2   gender               9700 non-null   object 
 3   department           9200 non-null   object 
 4   years_of_experience  9000 non-null   float64
 5   salary               9300 non-null   float64
 6   performance_score    9500 non-null   float64
 7   training_hours       8500 non-null   float64
 8   job_satisfaction     9000 non-null   float64
 9   last_promotion       8000 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 781.4+ KB


In [58]:
# Shape
df.shape

(10000, 10)

In [59]:
# check for duplicates
df.duplicated().any()

np.False_

In [60]:
# check for missing values
df.isnull().sum()

employee_id               0
age                     500
gender                  300
department              800
years_of_experience    1000
salary                  700
performance_score       500
training_hours         1500
job_satisfaction       1000
last_promotion         2000
dtype: int64

In [61]:
# missing value threshold
thresh = df.isnull().sum() * 100 / len(df)
thresh

employee_id             0.0
age                     5.0
gender                  3.0
department              8.0
years_of_experience    10.0
salary                  7.0
performance_score       5.0
training_hours         15.0
job_satisfaction       10.0
last_promotion         20.0
dtype: float64

In [62]:
# Step 1: Calculate missing value percentage per column
missing_percent = df.isnull().mean()
missing_percent

employee_id            0.00
age                    0.05
gender                 0.03
department             0.08
years_of_experience    0.10
salary                 0.07
performance_score      0.05
training_hours         0.15
job_satisfaction       0.10
last_promotion         0.20
dtype: float64

In [63]:
# column with 5% missing values
columns_to_clean = missing_percent[missing_percent <= 0.05].index
columns_to_clean

Index(['employee_id', 'age', 'gender', 'performance_score'], dtype='object')

In [64]:
# Drop rows with missing values in those columns
df = df.dropna(subset=columns_to_clean)
df

,employee_id,age,gender,department,years_of_experience,salary,performance_score,training_hours,job_satisfaction,last_promotion
1,2,46.0,Male,Sales,NaN,96192.0,7.0,30.0,3.0,0.0
2,3,32.0,Female,HR,12.0,45609.0,1.0,NaN,5.0,3.0
3,4,60.0,Female,Sales,12.0,49535.0,1.0,12.0,2.0,6.0
4,5,25.0,Female,NaN,4.0,34110.0,9.0,NaN,1.0,3.0
5,6,38.0,Female,Finance,11.0,139518.0,6.0,20.0,3.0,7.0
...,...,...,...,...,...,...,...,...,...,...
9994,9995,19.0,Female,Sales,1.0,94770.0,10.0,NaN,5.0,NaN
9995,9996,34.0,Female,Marketing,5.0,124380.0,1.0,15.0,4.0,NaN
9996,9997,47.0,Male,HR,7.0,31740.0,7.0,30.0,3.0,NaN
9998,9999,24.0,Male,NaN,20.0,98354.0,6.0,32.0,2.0,4.0


In [65]:
df.isnull().sum()

employee_id               0
age                       0
gender                    0
department              692
years_of_experience     851
salary                  613
performance_score         0
training_hours         1327
job_satisfaction        889
last_promotion         1763
dtype: int64

In [66]:
# drop ID column
df = df.drop(['employee_id'], axis=1)
df

,age,gender,department,years_of_experience,salary,performance_score,training_hours,job_satisfaction,last_promotion
1,46.0,Male,Sales,NaN,96192.0,7.0,30.0,3.0,0.0
2,32.0,Female,HR,12.0,45609.0,1.0,NaN,5.0,3.0
3,60.0,Female,Sales,12.0,49535.0,1.0,12.0,2.0,6.0
4,25.0,Female,NaN,4.0,34110.0,9.0,NaN,1.0,3.0
5,38.0,Female,Finance,11.0,139518.0,6.0,20.0,3.0,7.0
...,...,...,...,...,...,...,...,...,...
9994,19.0,Female,Sales,1.0,94770.0,10.0,NaN,5.0,NaN
9995,34.0,Female,Marketing,5.0,124380.0,1.0,15.0,4.0,NaN
9996,47.0,Male,HR,7.0,31740.0,7.0,30.0,3.0,NaN
9998,24.0,Male,NaN,20.0,98354.0,6.0,32.0,2.0,4.0


In [67]:
# convert gender and department to categorical variables
df['gender'] = df['gender'].astype('category')
df['department'] = df['department'].astype('category')

In [68]:
# Convert categorical variables to numeric
df['gender'] = df['gender'].cat.codes
df['department'] = df['department'].cat.codes
df.head()

,age,gender,department,years_of_experience,salary,performance_score,training_hours,job_satisfaction,last_promotion
1,46.0,1,4,NaN,96192.0,7.0,30.0,3.0,0.0
2,32.0,0,1,12.0,45609.0,1.0,NaN,5.0,3.0
3,60.0,0,4,12.0,49535.0,1.0,12.0,2.0,6.0
4,25.0,0,-1,4.0,34110.0,9.0,NaN,1.0,3.0
5,38.0,0,0,11.0,139518.0,6.0,20.0,3.0,7.0


In [69]:
# Split the data into training and testing sets
X_num = df.drop(['performance_score','gender','department'], axis=1).values
X_cat = df[['gender','department']].values
y = df['performance_score'].values

In [70]:
# split to test and train
# Numerical
X_train_num, X_test_num, y_train, y_test = train_test_split(X_num, y, test_size=0.2, random_state=42)

# Categorical
X_train_cat, X_test_cat, y_train, y_test = train_test_split(X_cat, y, test_size=0.2, random_state=42)

In [71]:
# Simple Imputer for numerical data
imp_num = SimpleImputer(strategy='mean')
X_train_num = imp_num.fit_transform(X_train_num)
X_test_num = imp_num.transform(X_test_num)

In [72]:
# Simple Imputer for categorical data
imp_cat = SimpleImputer(strategy='most_frequent')
X_train_cat= imp_cat.fit_transform(X_train_cat)
X_test_cat = imp_cat.transform(X_test_cat)

In [73]:
# Concat the transformed numerical and categorical data
X_train = np.concatenate((X_train_num, X_train_cat), axis=1)
X_test = np.concatenate((X_test_num, X_test_cat), axis=1)

Imputing within a pipeline

In [74]:
df['gender'] = np.where(df['gender'] == 'Male', 1, 0)
df['department'] = np.where(df['department'] == 'Sales', 1, 0)

In [75]:
# create X and Y
X = df.drop(['performance_score'], axis=1).values
y = df['performance_score'].values

In [76]:
# Build a pipeline
steps = [('imputer', SimpleImputer(),
         ('model', LogisticRegression()))]
pipeline = Pipeline(steps)

In [78]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# fit the pipeline model
pipeline.fit(X_train, y_train)

# score the model
pipeline.score(X_test, y_test)

ValueError: too many values to unpack (expected 2)